# Prep

This workbook reads in and pre-processes the titanic data into a Pandas dataset. The goal is to simply call in this workbook at the beginning of each other workbook to prevent redundant code. 

In [8]:
# Test variables
'''
embarked = 'integers-zero'
age = 'baby-zero-other-avg'
sex = 'integer'
fare = 'integer-floor'
'''

# Load variables from other notebook
%store -r

# Read In Data

Actually import the data into python and set an index


In [9]:
# Import relevant libraries

# Data manipulation
import pandas as pd
import numpy as np

# Ease of displaying multiple tables
from IPython.display import display, HTML

In [10]:
# Read in datasets 
titanic_df = pd.read_csv("data/train.csv")
titanic_test_df = pd.read_csv("data/test.csv")

In [11]:
# Set passengerid as index
titanic_df = titanic_df.set_index('PassengerId')
titanic_test_df = titanic_test_df.set_index('PassengerId')

#titanic_df.head()
titanic_df.shape

(891, 11)

# Feature Processing

Some of the columns aren't amazing. IE, missing values, categories, etc. For each column, we'll have a default value (the original) and the ability to let other notebooks send in a variable change.

Viewing the data above, it seems out of scope to process `Name`, `Ticket`, and `Cabin`. Perhaps that's another venture...

At the beginning of each section, look for...

**variable_name** = \[*'default'*, 'option-1', 'option-2',... \] 

... where the default option means do nothing.

## Missing Values

To start, let's see what imputation is required: 

In [12]:
# check missing values
display(titanic_df.isnull().sum())
display(titanic_test_df.isnull().sum())

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

...So, it's fair to say that `Age`, `Fare`, and `Embarked` need some thinking about. 

## Embarked 

**embarked** = \[*'CQS'*, 'integers-zero', 'integers-S', 'integers-C', 'integers-Q'\]

The Titanic was loaded in Southhampton ('S'), stopped at Cherbourg ('C'), then one final stop at Queenstown ('Q') (Source: https://www.thoughtco.com/titanic-timeline-1779210, accessed 2019-03-11).

There are two records with missing embarkation data in the training set. So, in addition to default, we can try to give those a new value (0), or just pretend they were in S, C, or Q. 

In [13]:
#print("Training:")
#display(titanic_df[['PassengerId', 'Embarked']].groupby(['Embarked']).agg(['count']))

99% populated with values C, Q and S. 

### Integers

When converting to ordinals, going to code 'S' => 1, 'C' => 2, 'Q' => 3, since that's the order the ship embarked

There's only two values in the training set that are missing, but let's still define what to do. Either set the NAN values to zero (`integers-zero`), or set them to one of the three embarkation ports (`integers-S|C|S`)

In [14]:
# We kinda hope embarked is defined... but if not, just don't change the column

try:
    if embarked == 'integers-zero':
        embarked_map = {np.nan: 0, 'S': 1, 'C': 2, 'Q': 3}
        titanic_df.replace({'Embarked': embarked_map}, inplace=True)
        titanic_test_df.replace({'Embarked': embarked_map}, inplace=True)
    elif embarked == 'integers-S':
        embarked_map = {np.nan: 1, 'S': 1, 'C': 2, 'Q': 3}
        titanic_df.replace({'Embarked': embarked_map}, inplace=True)
        titanic_test_df.replace({'Embarked': embarked_map}, inplace=True)
    elif embarked == 'integers-C':
        embarked_map = {np.nan: 2, 'S': 1, 'C': 2, 'Q': 3}
        titanic_df.replace({'Embarked': embarked_map}, inplace=True)
        titanic_test_df.replace({'Embarked': embarked_map}, inplace=True)
    elif embarked == 'integers-Q':
        embarked_map = {np.nan: 3, 'S': 1, 'C': 2, 'Q': 3}
        titanic_df.replace({'Embarked': embarked_map}, inplace=True)
        titanic_test_df.replace({'Embarked': embarked_map}, inplace=True)
except NameError:
    pass        

## Fare

**fare** = \[*'default'*, 'integer-round', 'integer-ceiling', 'integer-floor'\]

There's only one value missing in the test set for fare, and it's already a float64. Otherwise, there are a lot of 0 value fairs. I think the only option is default, let's turn the NAN fair to zero.

Additionally, there's a lot of complexity in a fare's precise value, maybe it makes sense to just convert it to integers? If the model calling this notebook wants it, we can optionally turn fare into an int64 column, which means every column is int64. Nice, right? 

In [15]:
# Fill NA with 0
titanic_test_df['Fare'] = titanic_test_df['Fare'].fillna(0)

In [16]:
# Encode fares as int64 if desired
try:
    if fare == 'integer-round':
        titanic_df.Fare = titanic_df.Fare.round().astype(np.int64)
        titanic_test_df.Fare = titanic_test_df.Fare.round().astype(np.int64)
    elif fare == 'integer-ceiling':
        titanic_df.Fare = titanic_df.Fare.apply(np.ceil).astype(np.int64)
        titanic_test_df.Fare = titanic_test_df.Fare.apply(np.ceil).astype(np.int64)
    elif fare == 'integer-floor':
        titanic_df.Fare = titanic_df.Fare.apply(np.floor).astype(np.int64)
        titanic_test_df.Fare = titanic_test_df.Fare.apply(np.floor).astype(np.int64)
except NameError:
    pass        

## Age 

**age** = \[*'default'*, 'baby-zero', 'baby-zero-other-avg', 'baby-default-other-avg'\]

Age is a tricky one... there are a lot of missing values. And not strictly integers, since if you're less than 1 year old, they provide a fraction. I think there are a few ways to handle it: 

* Get rid of float status by rounding down to zero for babies
* impute age as average age of passengers overall
* impute age as average age of passenger class (r = -0.37). 

..definitely a place to explore in the future. But let's just start with average age.

In [17]:
# Calculate average age
from math import floor

# just like with newborns, gonna assume if you're 29 and 250 days, you're still 29. 
avg_age = floor(titanic_df['Age'].mean(axis=0, skipna = True))

In [18]:
try:
    if age == 'baby-zero':
        titanic_df.loc[titanic_df["Age"] < 1, ["Age"]] = 0
        titanic_test_df.loc[titanic_test_df["Age"] < 1, ["Age"]] = 0
        titanic_df["Age"] = titanic_df["Age"].astype(np.int64)
        titanic_test_df["Age"] = titanic_test_df["Age"].astype(np.int64)
    elif age == 'baby-zero-other-avg':
        titanic_df.loc[titanic_df["Age"] < 1, ["Age"]] = 0
        titanic_df['Age'] = titanic_df['Age'].fillna(avg_age).astype(np.int64)
        titanic_test_df.loc[titanic_test_df["Age"] < 1, ["Age"]] = 0
        titanic_test_df['Age'] = titanic_test_df['Age'].fillna(avg_age).astype(np.int64)
        #titanic_df["Age"] = titanic_df["Age"].astype(np.int64)
        #titanic_test_df["Age"] = titanic_test_df["Age"].astype(np.int64)
    elif age == 'baby-default-other-avg':
        titanic_df['Age'] = titanic_df['Age'].fillna(avg_age).astype(np.int64)
        titanic_test_df['Age'] = titanic_test_df['Age'].fillna(avg_age).astype(np.int64)
        #titanic_df["Age"] = titanic_df["Age"].astype(np.int64)
        #titanic_test_df["Age"] = titanic_test_df["Age"].astype(np.int64)
except NameError:
    pass        

## Sex

**sex** = \[*'default'*, 'integer'\]

Every passenger had it's sex recorded... which is silly. But for modeling purposes, it might be nice to have "male" and "female" turned into 0 and 1. 


In [19]:
try:
    if sex == 'integer':
        sex_map = {'male': 0, 'female': 1}
        titanic_df.replace({'Sex': sex_map}, inplace=True)
        titanic_test_df.replace({'Sex': sex_map}, inplace=True)
except NameError:
    pass        

In [20]:
# prove everything worked
#display(titanic_df.isnull().sum())
#display(titanic_test_df.isnull().sum())
#display(titanic_df.head())
#display(titanic_test_df.head())
#display(titanic_df.dtypes)

## Create Train and Validate subsets

We don't technically have access to the full dataset, thus entries in test.csv should be ignored until we are ready to make a Kaggle submission. 

Let's do an 80/20 split of the train dataset into train and validate

In [21]:
from sklearn.model_selection import train_test_split

X = titanic_df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
Y = titanic_df[["Survived"]]

X_train, X_validate, y_train, y_validate = train_test_split(X, Y, test_size=0.2, random_state=10)

X_test = titanic_test_df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]


In [24]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
58,3,0,28,0,0,7,2
718,2,1,27,0,0,10,1
432,3,1,29,1,0,16,1
634,1,0,29,0,0,0,1
164,3,0,17,0,0,8,1


In [23]:
%store X_train
%store X_validate
%store y_train
%store y_validate
%store X_test

Stored 'X_train' (DataFrame)
Stored 'X_validate' (DataFrame)
Stored 'y_train' (DataFrame)
Stored 'y_validate' (DataFrame)
Stored 'X_test' (DataFrame)
